In [67]:
import numpy as np
import pickle
import random
import re
import spacy

from time import time

from src.simple_lookup import simple_lookup

In [2]:
data = pickle.load(open('notebooks/science-counts.pkl', 'rb'))

In [3]:
simple_lookup('propose', 'VERB', embkey="arxiv_abs")['words']

['introduce',
 'present',
 'develop',
 'devise',
 'proposes',
 'describe',
 'adopt',
 'employ',
 'introduces',
 'presents']

In [11]:
nouns = data['NOUN']
nouns_sorted = sorted(nouns, key=nouns.__getitem__, reverse=True)
num = len(nouns.keys())
print('num nouns:', num)
top50 = nouns_sorted[:50]
mid50 = nouns_sorted[1000:1050]
queries = random.sample(top50, 10) + random.sample(mid50, 10)
queries

num nouns: 36241


['dataset',
 'analysis',
 'optimization',
 'time',
 'systems',
 'system',
 'function',
 'classification',
 'problem',
 'accuracy',
 'curves',
 'neighbors',
 'fraction',
 'decades',
 'implications',
 'setup',
 'manifold',
 'abstraction',
 'pedestrian',
 'messages']

Attempt at specificity/generality measure just with frequency info.
Doesn't work that well.

In [22]:
print('specificy: +1 more specific than query, -1 less specific, 0 about the same')
print('{}\t{}\t{}\t{}\t{}'.format('query', 'response', 'specificy', 'q count', 'r count'))
for q in queries:
    responses = simple_lookup(q, 'NOUN', embkey="arxiv_abs")['words']
    for r in responses:
        d = nouns[q] - nouns[r]
        p = nouns[q]*.5
        s = 0
        if d > p:
            s = 1
        if d < -p:
            s = -1
        print('{}\t{}\t{}\t{}\t{}'.format(q, r, s, nouns[q], nouns[r]))

specificy: +1 more specific than query, -1 less specific, 0 about the same
query	response	specificy	q count	r count
dataset	database	1	6328	1610
dataset	datasets	0	6328	6321
dataset	imagenet	1	6328	4
dataset	treebank	1	6328	37
dataset	lfw	1	6328	1
dataset	wall	1	6328	61
dataset	databases	1	6328	733
dataset	suite	1	6328	132
dataset	digits	1	6328	136
dataset	mnist	1	6328	1
analysis	analyses	1	8275	377
analysis	investigation	1	8275	331
analysis	justification	1	8275	131
analysis	characterization	1	8275	428
analysis	understanding	1	8275	1339
analysis	visualization	1	8275	402
analysis	investigations	1	8275	100
analysis	studies	1	8275	1597
analysis	overview	1	8275	316
analysis	theory	1	8275	3096
optimization	optimisation	1	5856	320
optimization	minimization	1	5856	1074
optimization	solving	1	5856	33
optimization	relaxation	1	5856	414
optimization	nonconvex	1	5856	175
optimization	solver	1	5856	429
optimization	relaxations	1	5856	136
optimization	approximation	1	5856	2160
optimization	multiobj

In [41]:
with open('dat/corpora/arxiv_abs/summaries.txt', 'r') as fle:
    abstracts = fle.read()

abstracts_ = abstracts.split('\n')
print(len(abstracts_))
abstracts_[0]

82001


'We propose an architecture for VQA which utilizes recurrent layers to generate visual and textual attention. The memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question. Our single model outperforms the first place winner on the VQA 1.0 dataset, performs within margin to the current state-of-the-art ensemble model. We also experiment with replacing attention mechanisms in other state-of-the-art models with our implementation and show increased accuracy. In both cases, our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the VQA dataset.'

In [42]:
contexts = {}
count = 0
for a in abstracts_:
    count += 1
    if count % 1000 == 0:
        print('.', end=' ')
    clean = re.sub('[,.\?]', '', a)
    words = clean.lower().split(' ')
    for i, w in enumerate(words):
        if w not in contexts:
            contexts[w] = []
        for j in range(max(0, i-2), min(len(words), i+2)):
            if words[j] not in contexts[w]:
                contexts[w].append(words[j])

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [43]:
print(len(contexts['dataset']))
print(len(contexts['metadata']))

3353
204


In [66]:

print('specificy: +1 more specific than query, -1 less specific, 0 about the same')
print('{:<20}{:<20}{:<10}{:<10}{:<10}'.format('query', 'response', 'specificy', 'q count', 'r count'))
print('--------')
for q in ['network']:
    responses = simple_lookup(q, 'NOUN', embkey="arxiv_abs")['words']
    for r in responses:
        d = len(contexts[q]) - len(contexts[r])
        p = len(contexts[q])*.5
        s = 0
        if d > p:
            s = 1
        if d < -p:
            s = -1
        print('{:<20}{:<20}{:<10}{:<10}{:<10}'.format(q, r, s, len(contexts[q]), len(contexts[r])))

specificy: +1 more specific than query, -1 less specific, 0 about the same
query               response            specificy q count   r count   
--------
network             net                 1         4605      416       
network             networks            0         4605      3237      
network             nets                1         4605      303       
network             enquirer            1         4605      15        
network             cnn                 1         4605      1215      
network             architecture        1         4605      1744      
network             convnet             1         4605      223       
network             decoder             1         4605      400       
network             controller          1         4605      321       
network             autoencoder         1         4605      357       


In [80]:
nouns = data['NOUN']
nouns_sorted = sorted(nouns, key=nouns.__getitem__, reverse=True)
num = len(nouns.keys())
top50 = nouns_sorted[:50]
mid50 = nouns_sorted[1000:1050]
manyqueries = random.sample(nouns_sorted, 1000)

specific = []
general = []
neither = []
for q in manyqueries:
    try:
        contexts[q]
        responses = simple_lookup(q, 'NOUN', embkey="arxiv_abs")['words']
        for r in responses:
            contexts[r]
    except:
        continue
    s = 0
    g = 0
    n = 0
    for r in responses:
        d = len(contexts[q]) - len(contexts[r])
        p = len(contexts[q])*.25
        if d > p:
            s += 1
        elif d < -p:
            g += 1
        else:
            n += 1
    specific.append(s)
    general.append(g)
    neither.append(n)

for array in [specific, general, neither]:
    print(np.mean(array), np.std(array))

3.7755775577557755 2.8613385556052915
3.5181518151815183 2.86877474392947
2.514851485148515 1.901608810205782
